In [3]:
!pip install bpemb
!pip install transformers
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached bpemb-0.3.4-py3-none-any.whl (19 kB)
  Using cached sentencepiece-0.1.97-cp37-cp37m-win_amd64.whl (1.1 MB)


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.



Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.



  Using cached multiprocess-0.70.14-py37-none-any.whl (115 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
#from turtle import write_docstringdict
from nltk.tokenize import (word_tokenize,TreebankWordTokenizer)
import nltk
from nltk.corpus import stopwords

import numpy as np
from numpy import array
from numpy import argmax

import pandas as pd

#import pyarrow as pa
#import pyarrow.parquet as pq

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None

import pickle

from pathlib import Path

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(123)
from torch.utils.data import Dataset,DataLoader
#from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR, CyclicLR 
from torch.optim.lr_scheduler import LambdaLR

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from bpemb import BPEmb
#from .autonotebook import tqdm as notebook_tqdm

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizer
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import AutoModelForQuestionAnswering
from transformers import AutoConfig

import datasets
from datasets import DatasetDict, Dataset, load_dataset, load_metric

import math
from math import log

from typing import List, Tuple, AnyStr

import string

import io

import random

#from torchcrf import CRF

import matplotlib.pyplot as plt

from copy import deepcopy

import heapq

from functools import partial

from collections import defaultdict, OrderedDict

import re

%reload_ext autoreload
%autoreload 2
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\miaha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\miaha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")
print(device)

cuda


In [5]:
# For using colab
from google.colab import drive
drive.mount('/content/drive') # this will trigger permission prompts
path = '/content/drive/MyDrive/Deep Learning - NLP project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We have in total 6 pairs models (12 in total). For the following document one set of models can be selected and run. Since the procedures are for each model are identical just rerun the code with another pair of models. This would require restarting the kernel/clearning the memory since the training the models will use up allmost all memeory. 

In [5]:
# Selecting models
all_models = {'model_classification_BERT_en.pt':'bert-base-cased', 'model_extract_BERT_en.pt':'bert-base-cased',
        'model_classification_m_BERT_en.pt':'bert-base-multilingual-cased', 'model_extract_m_BERT_en.pt':'bert-base-multilingual-cased',
        'model_classification_m_BERT_ko.pt':'bert-base-multilingual-cased', 'model_extract_m_BERT_ko.pt': 'bert-base-multilingual-cased',
        'model_classification_roberta_en.pt':'roberta-base',   'model_extract_roberta_en.pt':'roberta-base', 
        'model_classification_m_roberta_en.pt':'xlm-roberta-base',  'model_extract_m_roberta_en.pt':'xlm-roberta-base',
        'model_classification_m_roberta_ko.pt':'xlm-roberta-base',  'model_extract_m_roberta_ko.pt':'xlm-roberta-base'}
model_name_class = list(all_models.keys())[8]
model_name_extract = list(all_models.keys())[9]
model_name_class, model_name_extract

('model_classification_m_roberta_en.pt', 'model_extract_m_roberta_en.pt')

The next step is to load the tokenizer from the belonging model. The fine-tuned models will be loaded, this is for evaluation of the models without needing to fine-tune them again. 

In [7]:
# Load Tokenzier from huggingface
tokenizer = AutoTokenizer.from_pretrained(all_models[model_name_class])
tokenizer.add_special_tokens({'bos_token': '|CLS|',
                                'eos_token': '|SEP|',
                                'pad_token': '<pad>',
                                'unk_token': '|UNK|'})

# Load fine-tuned models
model_class = torch.load(path + model_name_class).to(device)
model_extract = torch.load(path + model_name_extract).to(device)
model_extract.resize_token_embeddings(len(tokenizer))

3

## Preprocess data 

Before we start working with our models we need to prepare our data. We are using the [TyDi QA](https://huggingface.co/datasets/copenlu/answerable_tydiqa) dataset, before we start fine-tuning the model we will convert the data into SQuAD format. 

In [9]:
def preprocess(df1, df2 = None, exist = False, lan = 'english'):

    df_train = pd.concat([df1.drop(['annotations'], axis=1), df1['annotations'].apply(pd.Series)], axis=1)

    ## label 1 for answerable questions and 0 for unanswerable questions
    df_train['label'] = [1 if x != [-1] else 0 for x in df_train['answer_start'].values]
    
    ## select relevant feature columns 
    cols = ['question_text','document_plaintext','label','answer_start','answer_text']
    df_train = df_train[cols]
      
    df_train['text'] = df_train["question_text"] + " |SEP| " + df_train['document_plaintext'] 
 
    if exist: 
        df2[["answer_start_1","answer_text"]] = df2['annotations'].apply(pd.Series)

        df_val = pd.concat([df2.drop(['annotations'], axis=1), df2['annotations'].apply(pd.Series)], axis=1)

        ## label 1 for answerable questions and 0 for unanswerable questions
        df_val['label'] = [1 if x != [-1] else 0 for x in df_val['answer_start'].values]

        ## select relevant feature columns 
        df_val = df_val[cols]

        df_val['text'] = df_val["question_text"] + " |SEP| " + df_val['document_plaintext']

    if exist:
      return df_train.sample(frac=1).reset_index(drop=True), df_val.reset_index(drop=True)
    else:
      return df_train.sample(frac=1).reset_index(drop=True)

def create_squad(df_train, df_val):

    df_train_squad = df_train[df_train['label'] == 1].reset_index(drop=True)
    df_train_squad_full = df_train
    df_val_squad = df_val[df_val['label'] == 1].reset_index(drop=True)
    df_val_squad_full = df_val

    l_d_answers_t = []
    l_d_id_t = []
    for index, row in df_train_squad.iterrows():
        l_d_answers_t.append({'answer_start':row['answer_start'], 'text': row['answer_text']})
        l_d_id_t.append(f'THIS_IS_{index}_TEXT_IN_TRAIN')

    l_d_answers_t_f = []
    l_d_id_t_f = []
    for index, row in df_train_squad_full.iterrows():
        l_d_answers_t_f.append({'answer_start':row['answer_start'], 'text': row['answer_text']})
        l_d_id_t_f.append(f'THIS_IS_{index}_TEXT_IN_TRAIN')    

    l_d_answers_v = []
    l_d_id_v = []
    for index, row in df_val_squad.iterrows():
        l_d_answers_v.append({'answer_start':row['answer_start'], 'text': row['answer_text'][0]}) # Not sure why they are in an extra list
        l_d_id_v.append(f'THIS_IS_{index}_TEXT_IN_VAL_ANSWERABLE')

    l_d_answers_f = []
    l_d_id_v_f = []
    for index, row in df_val_squad_full.iterrows():
        l_d_answers_f.append({'answer_start':row['answer_start'], 'text': row['answer_text'][0]}) # Not sure why they are in an extra list
        l_d_id_v_f.append(f'THIS_IS_{index}_TEXT_IN_VAL_FULL')

    train_squad = pd.DataFrame({'answers':l_d_answers_t, 'context': df_train_squad['document_plaintext'], 
                        'question':df_train_squad['question_text'],'id': l_d_id_t})
    
    train_squad_full = pd.DataFrame({'answers':l_d_answers_t_f, 'context': df_train_squad_full['document_plaintext'], 
                        'question':df_train_squad_full['question_text'],'id': l_d_id_t_f})
    
    val_squad = pd.DataFrame({'answers':l_d_answers_v, 'context': df_val_squad['document_plaintext'], 
                        'question':df_val_squad['question_text'],'id': l_d_id_v})
    
    val_full_squad = pd.DataFrame({'answers':l_d_answers_f, 'context': df_val_squad_full['document_plaintext'], 
                        'question':df_val_squad_full['question_text'],'id': l_d_id_v_f})
    
    train_dataset_squad = Dataset.from_pandas(train_squad)
    train_dataset_squad_full = Dataset.from_pandas(train_squad_full)
    test_dataset_squad = Dataset.from_pandas(val_squad)
    full_test_dataset_squad = Dataset.from_pandas(val_full_squad)

    return datasets.DatasetDict({"train":train_dataset_squad, "train_full":train_dataset_squad_full,"validation":test_dataset_squad, "validation_full":full_test_dataset_squad})

def question_parag_combine(questions, paragraphs):
    """
    This function combines the questions and paragraphs into a single text
    Args:
        questions: list of questions
        paragraphs: list of paragraphs
    Returns:
        list of combined questions and paragraphs
    """
    training_data = []
    for index in range(len(questions)):
        training_data += ["|CLS| " + questions[index] + " |sep| " + paragraphs[index] + " |sep|"]
        
    return training_data

def load_data(df_train=None, df_val=None, path_to_file=None):
  if path_to_file is None or not Path(path_to_file).is_file():
    df_en_train, df_en_val = preprocess(df_train[df_train['language'] == 'english'], df2 = df_val[df_val['language'] == 'english'],exist = True)

    df_ko_train, df_ko_val = preprocess(df_train[df_train['language'] == 'korean'], df2 = df_val[df_val['language'] == 'korean'],exist = True, lan = 'korean')
    
    df_lan_dict = {'english': 
    {'train': df_en_train, 'val': df_en_val},
    'korean': 
    {'train': df_ko_train, 'val': df_ko_val}
    }

    df_lan_dict['english']['df_train'] = df_en_train
    df_lan_dict['english']['df_val'] = df_en_val

    df_lan_dict['korean']['df_train'] = df_ko_train
    df_lan_dict['korean']['df_val'] = df_ko_val

    #questions mark??? vvv
    df_en_train = df_en_train.reset_index().drop(columns='index')
    df_en_val = df_en_val.reset_index().drop(columns='index')

    df_ja_train = df_ko_train.reset_index().drop(columns='index')
    df_ja_val = df_ko_val.reset_index().drop(columns='index')

    df_lan_dict['english']['train'] = df_en_train
    df_lan_dict['english']['val'] = df_en_val

    df_lan_dict['korean']['train'] = df_ko_train
    df_lan_dict['korean']['val'] = df_ko_val

    for lan in ['english', 'korean']:
      df_lan_dict[lan]['train_dataset'] = Dataset.from_pandas(df_lan_dict[lan]['train'][['question_text','document_plaintext','label']])
      df_lan_dict[lan]['test_dataset'] = Dataset.from_pandas(df_lan_dict[lan]['val'][['question_text','document_plaintext','label']])
      df_lan_dict[lan]['dict_dataset'] = datasets.DatasetDict({"train": df_lan_dict[lan]['train_dataset'], "test": df_lan_dict[lan]['test_dataset']})

      d_q = df_lan_dict[lan]['train_dataset']['question_text']
      d_p = df_lan_dict[lan]['train_dataset']['document_plaintext']
      training_data = question_parag_combine(d_q, d_p)
      training_labels =  df_lan_dict[lan]['train_dataset']['label']

      d_q = df_lan_dict[lan]['test_dataset']['question_text']
      d_p = df_lan_dict[lan]['test_dataset']['document_plaintext']
      validation_data = question_parag_combine(d_q, d_p)
      validation_labels = df_lan_dict[lan]['test_dataset']['label']

      data_set = {}
      sets = [['train',training_data, training_labels], ['val',validation_data, validation_labels]]
      for meta in sets:
          data_set[meta[0]] = {}
          data_set[meta[0]]['text'] = []
          data_set[meta[0]]['label'] = []
          
          for ind, text in enumerate(meta[1]):
              data_set[meta[0]]['text'].append(text)
              data_set[meta[0]]['label'].append(meta[2][ind])
              
      data_set = DatasetDict({'train':Dataset.from_dict(data_set['train']),
                              'valid':Dataset.from_dict(data_set['val'])\
                          })
      df_lan_dict[lan]['data_set'] = data_set
    
    df_lan_dict['english']['train']['text'] = "|CLS| " + df_en_train["question_text"] + " |sep| " + df_en_train['document_plaintext'] + " |sep|"
    df_lan_dict['english']['val']['text'] = "|CLS| " + df_en_val["question_text"] + " |sep| " + df_en_val['document_plaintext'] + " |sep|"

    df_lan_dict['korean']['train']['text'] = "|CLS| " + df_ko_train["question_text"] + " |sep| " + df_ko_train['document_plaintext'] + " |sep|"
    df_lan_dict['korean']['val']['text'] = "|CLS| " + df_ko_val["question_text"] + " |sep| " + df_ko_val['document_plaintext'] + " |sep|"

    with open(path + 'df_lan_dict.pickle', 'wb') as handle:
      pickle.dump(df_lan_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    with open(path + 'df_lan_dict.pickle', 'rb') as handle:
      df_lan_dict = pickle.load(handle)
    
    return df_lan_dict
  else:
    with open(path_to_file, 'rb') as handle:
      df_lan_dict = pickle.load(handle)
    
    return df_lan_dict



In [10]:
## training set
df_train = pd.read_parquet(path + 'train-00000-of-00001-af2f3eaa87d1aa8b.parquet')

## validation set 
df_val = pd.read_parquet(path + 'validation-00000-of-00001-1f04eb244a33fa1b.parquet')

# Get english subset
df_en_train, df_en_val = preprocess(df_train[df_train['language'] == 'english'],
                                        df2 = df_val[df_val['language'] == 'english'], exist = True, lan = 'english')
en_dataset_squad = create_squad(df_en_train, df_en_val)

# Get korean subset
df_ko_train, df_ko_val = preprocess(df_train[df_train['language'] == 'korean'],
                                        df2 = df_val[df_val['language'] == 'korean'], exist = True, lan = 'korean')
ko_dataset_squad = create_squad(df_ko_train, df_ko_val)

In [11]:
df_ko_val.groupby('label').count()['text']

label
0    309
1    308
Name: text, dtype: int64

In [12]:
ko_dataset_squad

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'question', 'id'],
        num_rows: 1625
    })
    train_full: Dataset({
        features: ['answers', 'context', 'question', 'id'],
        num_rows: 3249
    })
    validation: Dataset({
        features: ['answers', 'context', 'question', 'id'],
        num_rows: 308
    })
    validation_full: Dataset({
        features: ['answers', 'context', 'question', 'id'],
        num_rows: 617
    })
})

In [13]:
# Load another preprocessed data which in a dataframe format. 
with open(path+'df_lan_dict.pickle', 'rb') as handle:
  df_lan_dict = pickle.load(handle)

## Classification model 

### Fine-tune classification model 

First we define our metrics which the model will be evaluted on we have selected f1-score. 

In [14]:
from datasets import load_metric
metric = load_metric('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions.flatten(), references=labels.flatten(),average='weighted')

In [18]:
# Define the training procedure
def model_gpt2_train(dict, lan, n_epochs=1, model_gpt2 = None):

    # Load model form huggingface if no input model found
    if model_gpt2 is None:
        
        model_gpt2 = AutoModelForSequenceClassification.from_pretrained(all_models[model_name_class], num_labels = 2).to(device)
        model_gpt2.resize_token_embeddings(len(tokenizer))
    
    def tokenize_function(examples):
      return tokenizer(examples["text"], max_length = 512, padding='max_length',truncation=True)

    tydiqa_tok_splits = dict[lan]['data_set']

    tydiqa_tok_lm = tydiqa_tok_splits.map(tokenize_function, batched=True)

    # Select training arguments
    training_args = TrainingArguments(
        "test-clm",
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        weight_decay=0.01,
        num_train_epochs=n_epochs
    )

    # Define trainer
    trainer = Trainer(
        model=model_gpt2,
        args=training_args,
        train_dataset=tydiqa_tok_lm['train'],
        eval_dataset=tydiqa_tok_lm['valid'],
        compute_metrics=compute_metrics)

    eval_results = trainer.evaluate()
    print(f"Perplexity before training: {math.exp(eval_results['eval_loss']):.2f}")

    # Training the model
    trainer.train()

    eval_results = trainer.evaluate()

    print(f"Perplexity after training: {math.exp(eval_results['eval_loss']):.2f}")
    path_file = path + 'model_classification_m_roberta_' + lan[0:2] + '.pt'

    # Saving the model 
    torch.save(model_gpt2, path_file)
    return torch.load(path_file)

In [19]:
model_gpt2_train(df_lan_dict, lan='english', n_epochs=1)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 990
  Batch size = 8


...

Perplexity before training: 2.01


Epoch,Training Loss,Validation Loss,F1
1,0.548600,0.473495,0.819481


Saving model checkpoint to test-clm/checkpoint-500
Configuration saved in test-clm/checkpoint-500/config.json
Model weights saved in test-clm/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 990
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 990
  Batch size = 8


Perplexity after training: 1.61


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250005, 768)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [32]:
model_gpt2_train(df_lan_dict, lan='korean', n_epochs=1)

### Evaluation of fine-tunning classification model

We start of with definning some helper functions for evaluating our model. 

In [23]:
def accuracy(target, pred):
    return metrics.accuracy_score(target, pred)

def F1(target, pred):
    return metrics.f1_score(target, pred)

def make_target(label):
    return torch.tensor(label, dtype=torch.long)
    
def evaluate(model: nn.Module, df_dict: dict, lan, loss_function = nn.CrossEntropyLoss()):
  """
  Evaluates the model on the given dataset
  :param model: The model under evaluation
  :param valid_dl: A `DataLoader` reading validation data
  :return: The accuracy of the model on the dataset
  """
  # VERY IMPORTANT: Put your model in "eval" mode -- this disables things like 
  # layer normalization and dropout
  model.eval()
  logprob_all = []
  losses_all = []
  target_val = []
  predictions = []

  with torch.no_grad():
      for index, row in df_dict[lan]['val'].iterrows():
          instance_context = row['text']
          encoded = tokenizer(instance_context,
                      return_tensors='pt', truncation = True, padding=True, max_length = 512)
          encoded = {k:v.to(device) for k, v in encoded.items()}
          model_output = model(**encoded, output_hidden_states=True, return_dict=True)

          label = row['label']
          target = make_target(label)
          target_val.append(target.tolist())

          log_probs = model_output.logits[0] ## input CR
          #print([log_probs.softmax(dim=-1).detach().cpu().flatten().numpy()[0]])
          logprob_all.extend([log_probs.softmax(dim=-1).detach().cpu().flatten().numpy()[0]])

          #print(log_probs)

  return logprob_all

The evaluate function will compute the prediction of our models. By setting a threshold of 0.5 for the probabilities we will be able to compute the accuracy of our model based on the validation set. 

In [26]:
no_answer_prob_en = evaluate(model_class, df_lan_dict, 'english', loss_function = nn.CrossEntropyLoss())
accuracy(np.array(no_answer_prob_en)<0.5,df_lan_dict['english']['df_val']['label'])

0.8222222222222222

In [36]:
no_answer_prob_ko = evaluate(model_class, df_lan_dict, 'korean', loss_function = nn.CrossEntropyLoss())
accuracy(np.array(no_answer_prob_ko)<0.5,df_lan_dict['korean']['df_val']['label'])

0.8654781199351702

## Extraction model

### Fine-tuning extraction model

For the extraction model the fine-tuning code and method comes from [Hugging face's](https://huggingface.co/course/chapter7/7?fw=pt) course chapter 7. 

In [27]:
def get_train_features(tk, samples):
  '''
  Tokenizes all of the text in the given samples, splittling inputs that are too long for our model
  across multiple features. Finds the token offsets of the answers, which serve as the labels for
  our inputs.
  '''
  batch = tk.batch_encode_plus(
        [[q,c] for q,c in zip(samples['question'], samples['context'])], 
        padding='max_length', 
        truncation='only_second',
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True
    )

  # Get a list which maps the input features index to their original index in the 
  # samples list (for split inputs). E.g. if our batch size is 4 and the second sample
  # is split into 3 inputs because it is very large, sample_mapping would look like
  # [0, 1, 1, 1, 2, 3]
  sample_mapping = batch.pop('overflow_to_sample_mapping')
  # Get all of the character offsets for each token
  offset_mapping = batch.pop('offset_mapping')

  # Store the start and end tokens
  batch['start_tokens'] = []
  batch['end_tokens'] = []

  # Iterate through all of the offsets
  for i, offsets in enumerate(offset_mapping):
    # Get the right sample by mapping it to its original index
    sample_idx = sample_mapping[i]
    # Get the sequence IDs to know where context starts so we can ignore question tokens
    sequence_ids = batch.sequence_ids(i)

    # Get the start and end character positions of the answer
    ans = samples['answers'][sample_idx]
    start_char = ans['answer_start'][0]
    end_char = start_char + len(ans['text'][0])
    # while end_char > 0 and (end_char >= len(samples['context'][sample_idx]) or samples['context'][sample_idx][end_char] == ' '):
    #   end_char -= 1

    # Start from the first token in the context, which can be found by going to the 
    # first token where sequence_ids is 1
    start_token = 0
    while sequence_ids[start_token] != 1:
      start_token += 1

    end_token = len(offsets) - 1
    while sequence_ids[end_token] != 1:
      end_token -= 1

    # By default set it to the CLS token if the answer isn't in this input
    if start_char < offsets[start_token][0] or end_char > offsets[end_token][1]:
      start_token = 0
      end_token = 0
    # Otherwise find the correct token indices
    else:
      # Advance the start token index until we have passed the start character index 
      while start_token < len(offsets) and offsets[start_token][0] <= start_char:
        start_token += 1
      start_token -= 1
      
      # Decrease the end token index until we have passed the end character index
      while end_token >= 0 and offsets[end_token][1] >= end_char:
        end_token -= 1
      end_token += 1

    batch['start_tokens'].append(start_token)
    batch['end_tokens'].append(end_token)

  #batch['start_tokens'] = np.array(batch['start_tokens'])
  #batch['end_tokens'] = np.array(batch['end_tokens'])

  return batch

def collate_fn(inputs):
  '''
  Defines how to combine different samples in a batch
  '''
  input_ids = torch.tensor([i['input_ids'] for i in inputs])
  attention_mask = torch.tensor([i['attention_mask'] for i in inputs])
  start_tokens = torch.tensor([i['start_tokens'] for i in inputs])
  end_tokens = torch.tensor([i['end_tokens'] for i in inputs])

  # Truncate to max length
  max_len = max(attention_mask.sum(-1))
  input_ids = input_ids[:,:max_len]
  attention_mask = attention_mask[:,:max_len]
  
  return {'input_ids': input_ids, 'attention_mask': attention_mask, 'start_tokens': start_tokens, 'end_tokens': end_tokens}

In [28]:
def train(
    model: nn.Module, 
    train_dl: DataLoader, 
    optimizer: torch.optim.Optimizer, 
    scheduler: LambdaLR,
    n_epochs: int, 
    device: torch.device,
    lan: str
):
  """
  The main training loop which will optimize a given model on a given dataset
  :param model: The model being optimized
  :param train_dl: The training dataset
  :param optimizer: The optimizer used to update the model parameters
  :param n_epochs: Number of epochs to train for
  :param device: The device to train on
  """

  # Keep track of the loss and best accuracy
  losses = []

  # Iterate through epochs
  for ep in range(n_epochs):

    loss_epoch = []

    #Iterate through each batch in the dataloader
    for batch in tqdm(train_dl):
      # VERY IMPORTANT: Make sure the model is in training mode, which turns on 
      # things like dropout and layer normalization
      model.train()

      # VERY IMPORTANT: zero out all of the gradients on each iteration -- PyTorch
      # keeps track of these dynamically in its computation graph so you need to explicitly
      # zero them out
      optimizer.zero_grad()

      # Place each tensor on the GPU
      batch = {b: batch[b].to(device) for b in batch}

      # Pass the inputs through the model, get the current loss and logits
      outputs = model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          start_positions=batch['start_tokens'],
          end_positions=batch['end_tokens']
      )
      loss = outputs['loss']
      losses.append(loss.item())
      loss_epoch.append(loss.item())
      
      # Calculate all of the gradients and weight updates for the model
      loss.backward()
      # Optional: clip gradients
      #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Finally, update the weights of the model and advance the LR schedule
      optimizer.step()
      scheduler.step()
      #gc.collect()
    torch.save(model,'/content/drive/MyDrive/Deep Learning - NLP project/model_extract_m_roberta_' + lan[0:2] + '.pt')
    #torch.save(model,'/content/drive/MyDrive/Deep Learning - NLP project/model_RoBERTa_extract_' + lan[0:2] + '.pt')
  return losses

For the following sections of code, the english models are selected, but if the korean models are selected the comment code needs to be run instead. 

In [29]:
model_name_en = all_models[model_name_class]
model_en = AutoModelForQuestionAnswering.from_pretrained(model_name_en).to(device)
model_en.resize_token_embeddings(len(tokenizer))

# If it is the korean model selected
#model_ko = AutoModelForQuestionAnswering.from_pretrained(all_models[model_name_class]).to(device)
#model_ko.resize_token_embeddings(len(tokenizer))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f

Embedding(250005, 768)

In [30]:
tokenized_dataset_en = en_dataset_squad['train'].map(partial(get_train_features, tokenizer), batched=True, remove_columns=en_dataset_squad['train'].column_names)
train_dl_en = DataLoader(tokenized_dataset_en, collate_fn=collate_fn, shuffle=True, batch_size=4)

# tokenized_dataset_ko = ko_dataset_squad['train'].map(partial(get_train_features, tokenizer), batched=True, remove_columns=ko_dataset_squad['train'].column_names)
# train_dl_ko = DataLoader(tokenized_dataset_ko, collate_fn=collate_fn, shuffle=True, batch_size=4)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [31]:
# Create the optimizer
lr=3e-5
n_epochs = 5
weight_decay = 0.01
warmup_steps = 250

# optimizer = Adam(optimizer_grouped_parameters, lr=1e-3)
# scheduler = None
optimizer_en = AdamW(model_en.parameters(), lr=lr, betas=(0.9,0.98), eps=1e-6, weight_decay=weight_decay)
scheduler_en = get_linear_schedule_with_warmup(
    optimizer_en,
    warmup_steps,
    n_epochs * len(train_dl_en)
)

# optimizer_ko = AdamW(model_ko.parameters(), lr=lr, betas=(0.9,0.98), eps=1e-6, weight_decay=weight_decay)
# scheduler_ko = get_linear_schedule_with_warmup(
#     optimizer_ko,
#     warmup_steps,
#     n_epochs * len(train_dl_ko)
# )

We can now fine-tune our model

In [32]:
losses = train(
    model_en, 
    train_dl_en,
    optimizer_en, 
    scheduler_en,
    n_epochs, 
    device,
    'english'
)

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

In [ ]:
# Training for Korean extraction model
#  losses = train(
#     model_ko, 
#     train_dl_ko,
#     optimizer_ko, 
#     scheduler_ko,
#     n_epochs, 
#     device,
#     'korean'
# )

### Evaluate fine-tuned extraction model

Before we can evaluate the models we will define some helper functions. 

In [33]:
def get_validation_features(tk, samples):
  # First, tokenize the text. We get the offsets and return overflowing sequences in
  # order to break up long sequences into multiple inputs. The offsets will help us 
  # determine the original answer text 
  batch = tk.batch_encode_plus(
        [[q,c] for q,c in zip(samples['question'], samples['context'])], 
        padding='max_length', 
        truncation='only_second',
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True
    )
  
  # We'll store the ID of the samples to calculate squad score
  batch['example_id'] = []
  # The overflow sample map tells us which input each sample corresponds to
  sample_map = batch.pop('overflow_to_sample_mapping')

  for i in range(len(batch['input_ids'])):
    # The sample index tells us which of the values in "samples" these features belong to
    sample_idx = sample_map[i]
    sequence_ids = batch.sequence_ids(i)

    # Add the ID to map these features back to the correct sample
    batch['example_id'].append(samples['id'][sample_idx])

    #Set offsets for non-context words to be None for ease of processing
    batch['offset_mapping'][i] = [o if sequence_ids[k] == 1 else None for k,o in enumerate(batch['offset_mapping'][i])]

  return batch  

def val_collate_fn(inputs):
  input_ids = torch.tensor([i['input_ids'] for i in inputs])
  attention_mask = torch.tensor([i['attention_mask'] for i in inputs])

  # Truncate to max length
  max_len = max(attention_mask.sum(-1))
  input_ids = input_ids[:,:max_len]
  attention_mask = attention_mask[:,:max_len]
  
  return {'input_ids': input_ids, 'attention_mask': attention_mask}

In [34]:
validation_dataset_en = en_dataset_squad['validation_full'].map(partial(get_validation_features, tokenizer), batched=True, remove_columns=en_dataset_squad['validation_full'].column_names)
validation_dataset_ko = ko_dataset_squad['validation_full'].map(partial(get_validation_features, tokenizer), batched=True, remove_columns=ko_dataset_squad['validation_full'].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [35]:
def predict(model: nn.Module, valid_dl: DataLoader):
  """
  Evaluates the model on the given dataset
  :param model: The model under evaluation
  :param valid_dl: A `DataLoader` reading validation data
  :return: The accuracy of the model on the dataset
  """
  # VERY IMPORTANT: Put your model in "eval" mode -- this disables things like 
  # layer normalization and dropout
  model.eval()
  start_logits_all = []
  end_logits_all = []

  # ALSO IMPORTANT: Don't accumulate gradients during this process
  with torch.no_grad():
    for batch in tqdm(valid_dl, desc='Evaluation'):
      batch = {b: batch[b].to(device) for b in batch}

      # Pass the inputs through the model, get the current loss and logits
      outputs = model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask']
      )
      # Store the "start" class logits and "end" class logits for every token in the input
      start_logits_all.extend(list(outputs['start_logits'].detach().cpu().numpy()))
      end_logits_all.extend(list(outputs['end_logits'].detach().cpu().numpy()))


    return start_logits_all,end_logits_all

def post_process_predictions(examples, dataset, logits, num_possible_answers = 20, max_answer_length = 30):
  all_start_logits, all_end_logits = logits
  # Build a map from example to its corresponding features. This will allow us to index from
  # sample ID to all of the features for that sample (in case they were split up due to long input)
  example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
  features_per_example = defaultdict(list)
  for i, feature in enumerate(dataset):
      features_per_example[example_id_to_index[feature["example_id"]]].append(i)

  # Create somewhere to store our predictions
  predictions = OrderedDict()

  # Iterate through each sample in the dataset
  for j, sample in enumerate(tqdm(examples)):

    # Get the feature indices (all of the features split across the batch)
    feature_indices = features_per_example[j]
    # Get the original context which predumably has the answer text
    context = sample['context']

    preds = []
    # Iterate through all of the features
    for ft_idx in feature_indices:

      # Get the start and end answer logits for this input
      start_logits = all_start_logits[ft_idx]
      end_logits = all_end_logits[ft_idx]

      # Get the offsets to map token indices to character indices
      offset_mapping = dataset[ft_idx]['offset_mapping']

      # Sort the logits and take the top N
      start_indices = np.argsort(start_logits)[::-1][:num_possible_answers]
      end_indices = np.argsort(end_logits)[::-1][:num_possible_answers]

      # Iterate through start and end indices
      for start_index in start_indices:
        for end_index in end_indices:
          
          # Ignore this combination if either the indices are not in the context
          if start_index >= len(offset_mapping) or end_index >= len(offset_mapping) or offset_mapping[start_index] is None or offset_mapping[end_index] is None:
            continue

          # Also ignore if the start index is greater than the end index of the number of tokens
          # is greater than some specified threshold
          if start_index > end_index or end_index - start_index + 1 > max_answer_length:
            continue

          ans_text = context[offset_mapping[start_index][0]:offset_mapping[end_index][1]]
          preds.append({
              'score': start_logits[start_index] + end_logits[end_index],
              'text': ans_text
          })

    if len(preds) > 0:
      # Sort by score to get the top answer
      answer = sorted(preds, key=lambda x: x['score'], reverse=True)[0]
    else:
      answer = {'score': 0.0, 'text': ""}
          
    predictions[sample['id']] = answer['text']
  return predictions

In [36]:
compute_squadv2 = load_metric("squad_v2")

Before we move on, we found a small problem in hugging face's metric squad_v2, thus a small change was implemented by us.

In [37]:
# Evaluation metric
from datasets import load_metric
import collections
compute_squad = load_metric("squad")

## Official evaluation functions from huggingface, there was a small error which we fixed such it showed for with Ans and NoAns
# FROM https://github.com/huggingface/datasets/blob/df4bdd365f2abb695f113cbf8856a925bc70901b/metrics/squad_v2/evaluate.py#L105

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def make_qid_to_has_ans(dataset):
    qid_to_has_ans = {}
    for article in dataset:
        for p in article["paragraphs"]:
            for qa in p["qas"]:
                qid_to_has_ans[qa["id"]] = bool(qa["answers"]["text"])
    return qid_to_has_ans

def find_best_thresh(preds, scores, na_probs, qid_to_has_ans):
    num_no_ans = sum(1 for k in qid_to_has_ans if not qid_to_has_ans[k])
    cur_score = num_no_ans
    best_score = cur_score
    best_thresh = 0.0
    qid_list = sorted(na_probs, key=lambda k: na_probs[k])
    for i, qid in enumerate(qid_list):
        if qid not in scores:
            continue
        if qid_to_has_ans[qid]:
            diff = scores[qid]
        else:
            if preds[qid]:
                diff = -1
            else:
                diff = 0
        cur_score += diff
        if cur_score > best_score:
            best_score = cur_score
            best_thresh = na_probs[qid]
    return 100.0 * best_score / len(scores), best_thresh


def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans, na_prob_thresh):
    new_scores = {}
    for qid, s in scores.items():
        pred_na = na_probs[qid] > na_prob_thresh
        if pred_na:
            new_scores[qid] = float(not qid_to_has_ans[qid])
        else:
            new_scores[qid] = s
    return new_scores

def make_eval_dict(exact_scores, f1_scores, qid_list=None):
    if not qid_list:
        total = len(exact_scores)
        return collections.OrderedDict(
            [
                ("exact", 100.0 * sum(exact_scores.values()) / total),
                ("f1", 100.0 * sum(f1_scores.values()) / total),
                ("total", total),
            ]
        )
    else:
        total = len(qid_list)
        return collections.OrderedDict(
            [
                ("exact", 100.0 * sum(exact_scores[k] for k in qid_list) / total),
                ("f1", 100.0 * sum(f1_scores[k] for k in qid_list) / total),
                ("total", total),
            ]
        )

def merge_eval(main_eval, new_eval, prefix):
    for k in new_eval:
        main_eval[f"{prefix}_{k}"] = new_eval[k]

def find_all_best_thresh(main_eval, preds, exact_raw, f1_raw, na_probs, qid_to_has_ans):
    best_exact, exact_thresh = find_best_thresh(preds, exact_raw, na_probs, qid_to_has_ans)
    best_f1, f1_thresh = find_best_thresh(preds, f1_raw, na_probs, qid_to_has_ans)
    main_eval["best_exact"] = best_exact
    main_eval["best_exact_thresh"] = exact_thresh
    main_eval["best_f1"] = best_f1
    main_eval["best_f1_thresh"] = f1_thresh


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article["paragraphs"]:
            for qa in p["qas"]:
                qid = qa["id"]
                gold_answers = [qa["answers"]["text"]] # Changes made by us

                if not gold_answers:
                    # For unanswerable questions, only correct answer is empty string
                    gold_answers = [""]
                if qid not in preds:
                    print(f"Missing prediction for {qid}")
                    continue
                a_pred = preds[qid]
                # Take max over all gold answers
                exact_scores[qid] =  max(compute_exact(a[0], a_pred) for a in gold_answers) 
                f1_scores[qid] = max(compute_f1(a[0], a_pred) for a in gold_answers)
    return exact_scores, f1_scores

# FROM https://github.com/huggingface/datasets/blob/main/metrics/squad_v2/squad_v2.py
def compute_ans_unans(predictions, references, no_answer_threshold=1.0):
    no_answer_probabilities = {p["id"]: p["no_answer_probability"] for p in predictions}
    dataset = [{"paragraphs": [{"qas": references}]}]
    predictions = {p["id"]: p["prediction_text"] for p in predictions}

    qid_to_has_ans = make_qid_to_has_ans(dataset)  # maps qid to True/False
    has_ans_qids = [k for k, v in qid_to_has_ans.items() if v]
    no_ans_qids = [k for k, v in qid_to_has_ans.items() if not v]

    exact_raw, f1_raw = get_raw_scores(dataset, predictions)
    exact_thresh = apply_no_ans_threshold(exact_raw, no_answer_probabilities, qid_to_has_ans, no_answer_threshold)
    f1_thresh = apply_no_ans_threshold(f1_raw, no_answer_probabilities, qid_to_has_ans, no_answer_threshold)
    out_eval = make_eval_dict(exact_thresh, f1_thresh)

    if has_ans_qids:
        has_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=has_ans_qids)
        merge_eval(out_eval, has_ans_eval, "HasAns")
    if no_ans_qids:
        no_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=no_ans_qids)
        merge_eval(out_eval, no_ans_eval, "NoAns")
    find_all_best_thresh(out_eval, predictions, exact_raw, f1_raw, no_answer_probabilities, qid_to_has_ans)
    return dict(out_eval)

We will predict the answers to the question using our fine-tuned model on the validation set. Thereafter we would also add the classification model to produce the full result. 

In [8]:
val_dl_en = DataLoader(validation_dataset_en, collate_fn=val_collate_fn, batch_size=32)
logits_en = predict(model_extract, val_dl_en)
predictions_en = post_process_predictions(en_dataset_squad['validation_full'], validation_dataset_en, logits_en)
formatted_predictions_en = [{'id': k, 'prediction_text': v, 'no_answer_probability': 0} for k,v in predictions_en.items()]
for i in range(len(no_answer_prob_en)):
  formatted_predictions_en[i]['no_answer_probability'] = no_answer_prob_en[i]
  if no_answer_prob_en[i] > 0.5:
    formatted_predictions_en[i]['prediction_text'] = ''
gold_en = [{'id': example['id'], 'answers': example['answers']} for example in en_dataset_squad['validation_full']]


In [ ]:
compute_ans_unans(predictions = formatted_predictions_en, references = gold_en)

In [82]:
val_dl_ko = DataLoader(validation_dataset_ko, collate_fn=val_collate_fn, batch_size=32)
logits_ko = predict(model_extract, val_dl_ko)
predictions_ko = post_process_predictions(ko_dataset_squad['validation_full'], validation_dataset_ko, logits_ko)
formatted_predictions_ko = [{'id': k, 'prediction_text': v, 'no_answer_probability': 0} for k,v in predictions_ko.items()]
for i in range(len(no_answer_prob_ko)):
  formatted_predictions_ko[i]['no_answer_probability'] = no_answer_prob_ko[i]
  if no_answer_prob_ko[i] > 0.5:
    formatted_predictions_ko[i]['prediction_text'] = ''
gold_ko = [{'id': example['id'], 'answers': example['answers']} for example in ko_dataset_squad['validation_full']]

Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/617 [00:00<?, ?it/s]

In [83]:
compute_ans_unans(predictions = formatted_predictions_ko, references = gold_ko)

{'exact': 66.12641815235008,
 'f1': 69.59001858558287,
 'total': 617,
 'HasAns_exact': 66.12641815235008,
 'HasAns_f1': 69.59001858558287,
 'HasAns_total': 617,
 'best_exact': 66.12641815235008,
 'best_exact_thresh': 0.9832002,
 'best_f1': 69.59001858558287,
 'best_f1_thresh': 0.9832002}